In [16]:
#bring in relevant libraries
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
import datetime
from datetime import datetime, timedelta
import time
import httplib, urllib, json, locale
from urlparse import urlparse
import operator
import bson
from collections import OrderedDict
from pymongo import MongoClient
from bson.objectid import ObjectId

merchant_ids = []
#connecting to our db
connection = MongoClient(host = '')
db=connection.heroku_ckbhgr9b
merchant = db['Merchants']
for merchants in merchant.find():
     merchant_ids.append(str(merchants['_id']))

for MERCHANT_ID in merchant_ids:
    #eventually we'll want this to be brought in externally from mongo

    #object_id = 'ObjectId(\"' + MERCHANT_ID + '\")'
    doc1 = []

    for doc in merchant.find({'_id':ObjectId(MERCHANT_ID)}):
        doc1.append(doc)
    try:
        access_token = doc1[0]['pos_access_token']
        record=db['Reports']
        access_token = str(access_token)
        location_id = doc1[0]['location_ids'][0]
    except:
        continue



    # In addition to an Authorization header, requests to the Connect API should
    # include the indicated Accept and Content-Type headers.
    request_headers = {'Authorization': 'Bearer ' + access_token,
                   'Accept':        'application/json',
                   'Content-Type':  'application/json'}

    # Send a GET request to the ListLocations endpoint and obtain the response.
    connection = httplib.HTTPSConnection('connect.squareup.com')
    request_path = '/v1/'+location_id+'/items'
    connection.request('GET', request_path, '', request_headers)
    response = connection.getresponse()



    # Convert the returned JSON body into an array of locations you can work with.
    items_data = json.loads(response.read())

    cat_name = []
    cat_id = []
    item_name = []
    price_list = []

    print 0
    for q in items_data:
        try:
            cat_name.append(q['category']['name'])
        except:
            cat_name.append('')
        try:
            cat_id.append(q['category_id'])
        except:
            cat_id.append('')
        try:
            item_name.append(q['name'])
        except:
            item_name.append('')
        try:
            price_list.append(float(q['variations'][0]['price_money']['amount'])/100)
        except:
            price_list.append(np.nan)

    df2 = pd.DataFrame({'category_id':cat_id,'cat_name':cat_name, 'name':item_name, 'price':price_list})
    
    avg_price_df = df2.groupby(['category_id','cat_name']).mean()
    print avg_price_df.head()

0
                                         price
category_id              cat_name             
LKXBHHW2XJ5TVNEDTLYSZIAV Breakfast    6.280000
PUONII3Q2HWYYVOYQ47QVXOK Lunch       10.283333
TOA4MEL6G5IJZ6G5NAWH3BSH Happy Hour  10.000000
0
                         price
category_id cat_name          
                      6.456667
0
                         price
category_id cat_name          
                      9.222222
0
                                                        price
category_id                          cat_name                
                                                    45.500000
5289e7a0-7ab3-439d-9ae7-21b403fc78c5 Fried Chicken  10.250000
789ec544-c300-4bd4-ab0e-0689520f3dba Extra           0.750000
b79fc917-ca60-43a3-b7dd-ca0d0a1b7ba7 Combo          12.916667
b9e5d996-142e-453e-9f7a-c5ca617a969f Sides           5.400000
0
                                                   price
category_id                          cat_name           
                     